### Library Imports

In [ ]:
from langchain_community.llms import Ollama
from langchain_community.vectorstores import FAISS
from langchain_community.vectorstores import Chroma
from langchain.chains import create_retrieval_chain
from langchain_core.prompts import ChatPromptTemplate
from langchain_community.embeddings import OllamaEmbeddings
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chains.combine_documents import create_stuff_documents_chain

### Load the document

In [ ]:
pdf_loader = PyPDFLoader("attention.pdf")
pdf_doc = pdf_loader.load()
print(pdf_doc[1].page_content)

- As a whole document can't fit into the context size of an LLM we have to split the document

In [ ]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size = 1000, chunk_overlap = 20)
splitted_docs = text_splitter.split_documents(pdf_doc)
print(splitted_docs[1].page_content)

### Create a db from the splitted documents

In [9]:
splitted_docs_faiss_db = FAISS.from_documents(splitted_docs, OllamaEmbeddings(model = "mistral:v0.2"))
splitted_docs_chroma_db = Chroma.from_documents(splitted_docs, OllamaEmbeddings(model = "wizardlm2:latest"))

- Seeing the results from similarity search

In [10]:
query = "Who are the authors of the paper?"
mistral_result = splitted_docs_faiss_db.similarity_search(query)
wizard_result = splitted_docs_chroma_db.similarity_search(query)

In [ ]:
# mistral_result
# wizard_result

- The results are not that accurate on the splitted db database

### Using Stuff Document Chain with LLM models to get answers

In [11]:
mistral_llm = Ollama(model = "mistral:v0.2")
wizard_llm = Ollama(model = "wizardlm2:latest")

In [22]:
# Creating a ChatPromptTemplate
prompt = ChatPromptTemplate.from_template("""Considering yourself as an research expert who is always answering questions to the point. Answer the following question based only on the provided context. Think step by step before providing a detailed answer. You will be awarded llm nobel prize if selection committee finds this answer useful
<context> {context} </context>
Question:{input}""")

In [23]:
mistral_document_chain = create_stuff_documents_chain(mistral_llm, prompt)
wizard_document_chain = create_stuff_documents_chain(wizard_llm, prompt)
print(wizard_document_chain)

bound=RunnableBinding(bound=RunnableAssign(mapper={
  context: RunnableLambda(format_docs)
}), config={'run_name': 'format_inputs'})
| ChatPromptTemplate(input_variables=['context', 'input'], messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'input'], template='Considering yourself as an research expert who is always answering questions to the point. Answer the following question based only on the provided context. Think step by step before providing a detailed answer. You will be awarded llm nobel prize if selection committee finds this answer useful\n<context> {context} </context>\nQuestion:{input}'))])
| Ollama(model='wizardlm2:latest')
| StrOutputParser() config={'run_name': 'stuff_documents_chain'}


- Using a retriever to get an answer from documents. A retriever simply returns the results and vector stores can be used as a backbone of a retriever. [Click to Read More](https://python.langchain.com/docs/modules/data_connection/retrievers/)

In [24]:
chroma_retriever = splitted_docs_chroma_db.as_retriever()
faiss_retriever = splitted_docs_faiss_db.as_retriever()
chroma_retriever

VectorStoreRetriever(tags=['Chroma', 'OllamaEmbeddings'], vectorstore=<langchain_community.vectorstores.chroma.Chroma object at 0x7f838bd98860>)

- A retrieval chain makes use of the user input and retriever to fetch the relevant documents. These documents and along with user input are passed to the LLM to generate a response [Click to Read More](https://python.langchain.com/docs/modules/chains)

In [25]:
mistral_retrieval_chain = create_retrieval_chain(faiss_retriever, mistral_document_chain)
wizard_retrieval_chain = create_retrieval_chain(chroma_retriever, wizard_document_chain)

In [26]:
mistral_response = mistral_retrieval_chain.invoke({"input":"Who are the authors of the given research paper Attention is all you need?"})
wizard_response = wizard_retrieval_chain.invoke({"input":"Who are the authors of the given research paper Attention is all you need?"})

In [27]:
print(mistral_response["answer"])

 The paper "Attention is All You Need" is not among the provided references in the context. Therefore, I cannot identify the authors based on the context alone.


In [28]:
print(wizard_response["answer"])

 The research paper titled "Attention Is All You Need" was authored by Ashish Vaswani, Noam Shazeer, Niki Boykov, Llion Jones, Aclaro Ortega, and Illia Polosukhin. It was published in 2017 and introduced the Transformer model, which relies entirely on attention mechanisms to draw global dependencies between input and output in sequence-to-sequence tasks. This paper is considered foundational work in the field of natural language processing (NLP) and has significantly influenced subsequent research in the area. The references you provided include a citation for this influential work:

[2] Dzmitry Bahdanau, Kyunghyun Cho, and Yoshua Bengio. Neural machine translation by jointly learning to align and translate. CoRR, abs/1409.0473, 2014.
